<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

# Library import
We import all the required Python libraries

The non-default libries are networkX (https://networkx.org/) and py4cytoscape (https://py4cytoscape.readthedocs.io/, only necessary if you wish to view the results in Cytoscape). 

You can eith have skm-tools package installed, or as provided in the same repository as this notebook. 

In [1]:
# run settings
DEV = True # use local skm_tools (if false, assume skm_tools is installed or in PYTHONPATH)
RUN_CYTOSCAPE = False # causes a notebook error before running Cytoscape integration
RUN_PATH_EXTRACTION = False # causes a notebook error before running path extraction integration

In [2]:
from datetime import datetime
from collections import defaultdict

# networks!
import networkx as nx

# Pretty results!
from IPython.display import display, HTML

In [3]:
if DEV:
    # The following allows us to import from the skm-tools package folder, directly.
    # Note the relative path to the folder containing the "skm_tools" directory. 
    from importlib import reload
    import sys
    sys.path.insert(0,'../')

from skm_tools import load_networks, pss_utils

In [4]:
today = datetime.today().strftime('%Y.%m.%d'); today

'2025.03.10'

# Parameter definitions

In [5]:
# Which PSS projection to use (see below in "Load PSS")
PSS_PROJECTION = "standard" 
# PSS_PROJECTION = "exploded"

# PSS edit settings
KEEP_NODES_LABELS = [
    'Complex',
    'Metabolite',
    'PlantAbstract',
    'PlantCoding', 
    'PlantNonCoding', 
    'Process'
]
KEEP_HOMOLOGUE_SPECIES = [
    'ath', 'stu'
] # only valid when using PSS_PROJECTION = "standard" 

SIMPLIFY = True

REMOVE_DEADEND_COMPLEXES = True

REMOVE_AND_REWIRE_METBOLITES = True
METABOLITES_TO_KEEP = [
    'Ca2+',
    'SA',
    'GA',
    'cZ',
    'ET',
    'JA',
    'JA-Ile',
    'ROS',
    'O2',
    'Cu2+',
    'ABA',
    'SL',
    'IAA',
] # Only used if REMOVE_AND_REWIRE_METBOLITES = True

REMOVE_AND_REWIRE_COMPLEXES = True
COMPLEXES_TO_KEEP = None # Only used if REMOVE_AND_REWIRE_COMPLEXES = True

EXTRACT_LCC = True

REMOVE_SELF_LOOPS = True

REMOVE_DUPLICATED_BINDING = True # Only keep one od A --> B or A <-- B in binding interactions. This will affect directed path finding!

# File paths 

If the below files do not exist, this code will use the latest live PSS instance (from [skm.nib.si/downloads](https://skm.nib.si/downloads)). 
If you want to use a version already downloaded, adjust the below paths accordingly. 

<b style="color:red">Note:</b> 
`skm-tools` cannot download "restricted" versions of PSS. If you want to use the restriced versions, you will need to login to [skm.nib.si](https://skm.nib.si/)  and downloaded them to the provided paths

In [6]:
from pathlib import Path

In [7]:
base_dir = Path("./")
data_dir = base_dir / "data"
output_dir = base_dir / "output"

In [8]:
if PSS_PROJECTION == "exploded":
    pss_edge_path = data_dir / f"pss-dinar-edges-public-{today}.tsv"
    pss_node_path = data_dir / f"pss-dinar-nodes-public-{today}.tsv"
elif PSS_PROJECTION == "standard":
    pss_edge_path = data_dir / f"pss-rxn-edges-public-{today}.tsv"
    pss_node_path = data_dir / f"pss-rxn-nodes-public-{today}.tsv"
else:
    print("STOP! Not a recognised format for this Notebook.")
    assert False

In [9]:
print("Checking files...")
print(f" Edge file ({pss_edge_path.absolute()}) exists: {pss_edge_path.exists()}")
print(f" Node file ({pss_node_path.absolute()}) exists: {pss_node_path.exists()}")

Checking files...
 Edge file (/home/cbleker/research/NIB/skm/skm-tools/examples/data/pss-rxn-edges-public-2025.03.10.tsv) exists: True
 Node file (/home/cbleker/research/NIB/skm/skm-tools/examples/data/pss-rxn-nodes-public-2025.03.10.tsv) exists: True


# Load PSS to networkx objects

PSS as a network comes in two projections:
* Standard projection "PSS-RXN" - functional clusters are nodes, conneted by edges representing reactions
* Exploded projection "DiNAR" - genes are nodes (edges representing reactions are duplcated for each gene in a functional cluster)

In [10]:
print(PSS_PROJECTION)

standard


Use appropriate functions of `skm-tools`

In [11]:
if PSS_PROJECTION == "exploded":
    g = load_networks.pss_dinar_to_networkx(
        edge_path=pss_edge_path, 
        node_path=pss_node_path
    )

elif PSS_PROJECTION == "standard":
    g = load_networks.pss_to_networkx(
        edge_path=pss_edge_path, 
        node_path=pss_node_path
    )

print(f"\nNumber of nodes: {g.number_of_nodes()}\nNumber of edges: {g.number_of_edges()}")


Number of nodes: 688
Number of edges: 1405


# Filtering

PSS contains nodes or processes that are not always relevent, such as viral proteins (Foreign nodes), Processes, Abiotic factors, or FunctionalClusters that do not include functional homologues of species of interest. 

Here, we show how to remove them from the network. 

In [12]:
# see the types of nodes present
set([data['node_type'] for n, data in g.nodes(data=True)])

{'Complex',
 'Condition',
 'ForeignAbiotic',
 'ForeignCoding',
 'ForeignEntity',
 'ForeignNonCoding',
 'Metabolite',
 'PlantAbstract',
 'PlantCoding',
 'PlantNonCoding',
 'Process'}

In [13]:
# Define the types we want to keep
KEEP_NODES_LABELS

['Complex',
 'Metabolite',
 'PlantAbstract',
 'PlantCoding',
 'PlantNonCoding',
 'Process']

In [14]:
# Define the species we want to keep
if PSS_PROJECTION != "standard":
    KEEP_HOMOLOGUE_SPECIES = None
KEEP_HOMOLOGUE_SPECIES

['ath', 'stu']

In [15]:
# Use the util function to filter PSS
removed = pss_utils.filter_pss_nodes(
    g, 
    node_types=KEEP_NODES_LABELS, 
    species=KEEP_HOMOLOGUE_SPECIES, 
    remove_isolates=True
)

Removed 91 nodes from network.


You can inspect the `removed` object to see the nodes that were removed, 
and the reasons for the removal

In [16]:
display(HTML(
    "<div style='height: 400px;'>" + \
     "<br>".join([f"<b>{name}:</b> {reason}" for name, reason in removed.items()]) +\
     "</div>"
))

In [17]:
print(f"\nNumber of nodes: {g.number_of_nodes()}\nNumber of edges: {g.number_of_edges()}")


Number of nodes: 597
Number of edges: 1178


# Simplifying

If multiple reactions had the (some of) same particiapnts (e.g. same enzyme and substrate, but different products, two seperate binding reactions), edge multiplication occurs (hence we are using a <b>Multi</b>`DiGraph`). Here, collapse these edges. 

In collapsing edges, the edge attribures have to be merged: all `reaction_id`s are kept, for other edge attributes a single value is kept, and a warning given if multiple non-unique values were observed. At the moment there is no logic to which value is kept. 

<b style="color:red">Note:</b> 
Unlike many of the other functions here, `pss_utils.simplify_pss` creates a __new__ graph object, and therefore needs to be assigned to a new variable.

In [18]:
if SIMPLIFY:
    g = pss_utils.simplify_pss(g)

rx00569,rx00610 --> interaction_type: Multiple values in ['activation', 'inhibition'].
	Keeping: inhibition.
rx00569,rx00610 --> reaction_type: Multiple values in ['protein activation', 'transcriptional/translational repression'].
	Keeping: protein activation.
rx00569,rx00610 --> reaction_effect: Multiple values in ['activation', 'inhibition'].
	Keeping: inhibition.
rx00569,rx00610 --> source_edge_type: Multiple values in ['ACTIVATES', 'INHIBITS'].
	Keeping: INHIBITS.
rx00569,rx00610 --> source_form: Multiple values in ['protein_active', 'protein'].
	Keeping: protein.
rx00569,rx00610 --> target_edge_type: Multiple values in ['PRODUCT', 'SUBSTRATE'].
	Keeping: SUBSTRATE.
rx00569,rx00610 --> target_location: Multiple values in ['cytoplasm', 'nucleus'].
	Keeping: cytoplasm.
rx00569,rx00610 --> target_form: Multiple values in ['protein_active', 'gene'].
	Keeping: gene.
rx00830,rx00831,rx00832,rx00833 --> target_location: Multiple values in ['nucleus', 'nucleus', 'nucleolus', 'nucleus'].
	K

In [19]:
print(f"\nNumber of nodes: {g.number_of_nodes()}\nNumber of edges: {g.number_of_edges()}")


Number of nodes: 597
Number of edges: 1145


# Remove deadend complexes

Complexes are created automatically if a binding/oligomerisation is entered into PSS. However, sometimes they only make sense to include here if the complex has some downstream interactions it takes part in, since this projection already includes the substrate-substrate interaction. If they do not have downstream interactors, we call then "deadend". 

The DiNAR projection does not include complexes if the binding/oligomerisation reaction if formulated as inhibition of the substrate(s) (and not activation of the complex). This removes the majority of the problematic complexes. But since (1) the inhibition or activation annotation of the reaction is not always included in the reaction formulation correctlyand (2) somebody maybe hasn't yet added the downstream interactions, let's check for deadend complexes and remove them anyway. 

In [20]:
if REMOVE_DEADEND_COMPLEXES:
    removed_complexes = pss_utils.remove_deadend_complexes(g)
    print(", ".join(removed_complexes))
print(f"\nNumber of nodes: {g.number_of_nodes()}\nNumber of edges: {g.number_of_edges()}")

Number of complexes removed: 63
EDS1|MPK3|PAD4, CRT|ETR, HSP90|RAR1|SGT1, NDR1|RIN4, RANGAP|Rx, GPAphid2|RANGAP, OBE1|WRKY17, OBE1|WRKY11, CO|OBE1, MKS1|WRKY33, WRKY30|WRKY53, D14|MAX2|SCF, EIN3(like)|JAZ, DELLA|GA|GID1, RISC - vsiRNA, SARD1|TCP8, TCP8|WRKY28, NAC019|TCP8, NPR1|WRKY18, CDK|NPR1, CDK|WRKY6,18, CDK|TGA2,5,6, BIK1|PEPR1|PEP1, ET|ETR, RISC - miR390, GAI|GI, GI|PIF4, CDK|RAP2-6, RAP2-6|SNRK2, CML12|PID, PBP1|PID, CMI1|ICR1, FKBP42|HSP90, FKBP42|CAM, CBL1|CIPK1, CBL9|CIPK1, FKBP62|HSP90.1, CBL1|CIPK23, CBL9|CIPK23, ABF1|IDD14, ABF2|IDD14, ABF3|IDD14, ABF4|IDD14, HSFA1d|PIF4, CBL2|CIPK3|Ca2+, CBL2|CIPK9|Ca2+, CBL2|CIPK26|Ca2+, CBL3|CIPK3|Ca2+, CBL3|CIPK9|Ca2+, CBL3|CIPK26|Ca2+, PR1|cholesterol, CBL2|CIPK3, CBL2|CIPK9, CBL2|CIPK23, CBL2|CIPK26, CBL3|CIPK3, CBL3|CIPK9, CBL3|CIPK23, CBL3|CIPK26, PIF3-TOC1-SP6A complex, CBL4|Ca2+, CBL4|CIPK24, GID|SCF

Number of nodes: 534
Number of edges: 1015


# Remove "uninteresting" Metabolites and Complexes, and rewire neighbours

In [21]:
all_metabolites = [n for n, data in g.nodes(data=True) if data["node_type"] == "Metabolite"]
display(HTML("<div><b>All metabolites: </b>" + ", ".join(all_metabolites) + "</div>"))
if METABOLITES_TO_KEEP is None:
    display(HTML("<div><b>Metabolites to keep: </b> None</div>"))
else:
    display(HTML("<div><b>Metabolites to keep: </b>" + ", ".join(METABOLITES_TO_KEEP) + "</div>"))

In [22]:
all_complexes = [n for n, data in g.nodes(data=True) if data["node_type"] == "Complex"]
display(HTML("<div><b>All complexes: </b>" + ", ".join(all_complexes) + "</div>"))

if COMPLEXES_TO_KEEP is None:
    display(HTML("<div><b>Complexes to keep: </b> None</div>"))
else:
    display(HTML("<div><b>Complexes to keep: </b>" + ", ".join(COMPLEXES_TO_KEEP) + "</div>"))

In [23]:
nodes_to_remove_and_rewire = set()

if REMOVE_AND_REWIRE_METBOLITES:
    if METABOLITES_TO_KEEP:
        nodes_to_remove_and_rewire.update(set(all_metabolites) - set(METABOLITES_TO_KEEP))
    else:
        nodes_to_remove_and_rewire.update(set(all_metabolites))

if REMOVE_AND_REWIRE_COMPLEXES:
    if COMPLEXES_TO_KEEP:
        nodes_to_remove_and_rewire.update(set(all_complexes) - set(COMPLEXES_TO_KEEP))
    else:
        nodes_to_remove_and_rewire.update(set(all_complexes))

display(HTML("<div><b>Nodes to remove and rewire: </b>" + ", ".join(nodes_to_remove_and_rewire) + "</div>"))

In [24]:
pss_utils.remove_and_rewire(g, nodes_to_remove_and_rewire)

iP-riboside
  Downstream: iP-ribotide -- To be removed
  Downstream ADK[AT3G09820,AT5G03300]
  Downstream: iP -- To be removed
     --->
  Upstream: iP -- To be removed
  Upstream: iP-ribotide -- To be removed

DZ-riboside
  Downstream: DZ-ribotide -- To be removed
  Downstream ADK[AT3G09820,AT5G03300]
  Downstream: DZ -- To be removed
     --->
  Upstream: DZ-ribotide -- To be removed
  Upstream: DZ -- To be removed

prenyl-tRNA
  Downstream: cis-prenyl-tRNA -- To be removed
     --->
  Upstream: DMAPP -- To be removed
  Upstream: tRNA-adenine -- To be removed
  Upstream IPT2,9[AT2G27760,AT5G20040]

tZ-riboside
  Downstream: tZ-ribotide -- To be removed
  Downstream ADK[AT3G09820,AT5G03300]
  Downstream: tZ -- To be removed
     --->
  Upstream: tZ -- To be removed
  Upstream: tZ-ribotide -- To be removed

NPR1|NPR1
  Downstream: NPR1[AT1G64280] -- Same source/target
     --->
  Upstream: NPR1[AT1G64280] -- Same source/target

9-cis-10&prime;-apo-&beta;-carotenal
  Downstream: CL -- T

In [25]:
print(f"\nNumber of nodes: {g.number_of_nodes()}\nNumber of edges: {g.number_of_edges()}")


Number of nodes: 386
Number of edges: 712


# Extract largest connected component (LCC)

In [26]:
for i, cc in enumerate(nx.weakly_connected_components(g)):
    print(f"Connected component {i:2}: {len(cc)}")

Connected component  0: 364
Connected component  1: 3
Connected component  2: 2
Connected component  3: 1
Connected component  4: 2
Connected component  5: 1
Connected component  6: 1
Connected component  7: 3
Connected component  8: 3
Connected component  9: 1
Connected component 10: 3
Connected component 11: 2


Keep the largest :

In [27]:
if EXTRACT_LCC:
    g = g.subgraph(max(nx.weakly_connected_components(g), key=len)).copy()

In [28]:
print(f"\nNumber of nodes: {g.number_of_nodes()}\nNumber of edges: {g.number_of_edges()}")


Number of nodes: 364
Number of edges: 697


# Remove self loops, and doubled binding edges

In [29]:
if REMOVE_SELF_LOOPS:
    g.remove_edges_from(nx.selfloop_edges(g))

In [30]:
print(f"\nNumber of nodes: {g.number_of_nodes()}\nNumber of edges: {g.number_of_edges()}")


Number of nodes: 364
Number of edges: 696


In [31]:
if REMOVE_DUPLICATED_BINDING:
    pss_utils.remove_duplicated_binding_edges(g)

Removed 72 edges from network.


In [33]:
print(f"\nNumber of nodes: {g.number_of_nodes()}\nNumber of edges: {g.number_of_edges()}")


Number of nodes: 364
Number of edges: 624


# Exporting graph to file

NetworkX does not have  the greatest export formats, so  making a Pandas dataframe and saving that seems the best. 

In [34]:
df = nx.to_pandas_edgelist(g)
df.head()

,source,target,source_edge_type,reaction_id,source_location,directed,source_form,reaction_effect,target_form,target_edge_type,reaction_type,interaction_type,target_location,note
0,"CRT[AT1G08450,AT1G09210,AT1G56340]","ETR[AT1G04310,AT1G66340,AT2G40940,AT3G04580,AT...",SUBSTRATE,rx00228,cytoplasm,False,protein_active,activation,protein,SUBSTRATE,binding/oligomerisation,binding,endoplasmic reticulum,NaN
1,"CRT[AT1G08450,AT1G09210,AT1G56340]",Ca2+,SUBSTRATE,rx00194,cytoplasm,False,protein_active,inhibition,metabolite,SUBSTRATE,binding/oligomerisation,inhibition,cytoplasm,NaN
2,OSCA1[AT4G04340],Ca2+,ACTIVATES,rx00603,cytoplasm,True,protein_active,activation,metabolite,TRANSLOCATE_TO,translocation,translocation,cytoplasm,NaN
3,"SNRK2[AT1G10940,AT1G78290,AT4G33950,SOTUB02G03...","AREB/ABF[AT1G45249,AT1G49720,AT3G19290,AT4G34000]",INHIBITS,"rx00569,rx00610",cytoplasm,True,protein,inhibition,gene,SUBSTRATE,protein activation,inhibition,cytoplasm,NaN
4,"SNRK2[AT1G10940,AT1G78290,AT4G33950,SOTUB02G03...","ARR-A[AT1G10470,AT1G19050,AT1G59940,AT1G74890,...",ACTIVATES,rx00570,cytoplasm,True,protein_active,activation,protein,SUBSTRATE,protein activation,activation,nucleus,NaN


In [35]:
df.to_csv(output_dir / f"pss-{PSS_PROJECTION}-refined-edgelist-{today}.tsv", sep="\t", index=None)

# Cytoscape 

In [36]:
# don't go beyond here with Run All
assert RUN_CYTOSCAPE

First open the Cytoscape application. Then the following cell will load the required library and and make sure you can connect to the Cytoscape application. 

More py4cytoscape documentation is here: https://py4cytoscape.readthedocs.io/

In [37]:
import py4cytoscape as p4c
p4c.cytoscape_ping()
p4c.cytoscape_version_info()

You are connected to Cytoscape!


{'apiVersion': 'v1',
 'cytoscapeVersion': '3.10.2',
 'automationAPIVersion': '1.9.0',
 'py4cytoscapeVersion': '1.9.0'}

Some helper functions for Cytoscape from skm-tools:

In [38]:
from skm_tools import cytoscape_utils

We set the Cytoscape collection name for this notebook. 

In [39]:
COLLECTION = f"PSS projection ({today})"
COLLECTION

'PSS projection (2025.03.10)'

## Load the PSS network into Cytoscape

We load the network, set a visual style, and apply the CoSE layout.

With skm-tools, we provide a default style for PSS, colouring the nodes by pathway.

Returned is the ID of the network view in Cytoscape.

In [40]:
pss_network_suid = p4c.networks.create_network_from_networkx(g, title="PSS", collection=COLLECTION)

Applying default style...
Applying preferred layout


In [41]:
# Apply a style
cytoscape_utils.apply_builtin_style(pss_network_suid, 'pss')
pss_network_suid

Applied PSS-default to 90162


90162

# Saving the node positions from Cytoscape

In [40]:
df_xy = p4c.get_node_position()
df_xy.to_csv(output_dir / f"pss-{PSS_PROJECTION}-refined-xy-coords-{today}.tsv", sep="\t")

# Extracting some paths

In [41]:
# don't go beyond here with Run All
assert RUN_PATH_EXTRACTION

AssertionError: 

# Example path extraction

First we identify the nodes of interest.

In [ ]:
JA = [x for x,y in g.nodes(data=True) if y['name']=="JA"][0]
SA = [x for x,y in g.nodes(data=True) if y['name']=="SA"][0];
ROS = [x for x,y in g.nodes(data=True) if y['name']=="ROS"][0]

print(JA)
print(SA)
print(ROS)

## JA --> SA + JA --> ROS

In [ ]:
JA_paths = [p for p in nx.all_shortest_paths(g, source=JA, target=SA)]
JA_paths += [p for p in nx.all_shortest_paths(g, source=JA, target=ROS)]

## SA --> JA + SA --> ROS

In [ ]:
SA_paths = [p for p in nx.all_shortest_paths(g, source=SA, target=JA)]
SA_paths += [p for p in nx.all_shortest_paths(g, source=SA, target=ROS)]

## ROS --> JA + ROS --> SA

In [ ]:
ROS_paths = [p for p in nx.all_shortest_paths(g, source=ROS, target=JA)]
ROS_paths += [p for p in nx.all_shortest_paths(g, source=ROS, target=SA)]

# Visualise paths in Cytoscape

Now we're going to highlight the paths we identified in the network by applying style bypasses.



We set the colours here:

In [ ]:
JA_COLOUR = "#66a61e"
SA_COLOUR = "#34858d"
ROS_COLOUR = "#dc1c1c"

We don't want to recolour already highlighted path elements, so we keep track of them here:

In [ ]:
done_nodes, done_edges = [], []

In [ ]:
for p in ROS_paths:
    done_nodes_now, done_edges_now = cytoscape_utils.highlight_path(p, ROS_COLOUR, skip_nodes=done_nodes, skip_edges=done_edges)
    done_nodes += done_nodes_now
    done_edges += done_edges_now
# Note - need to improve edge colouring

In [ ]:
for p in JA_paths:
    done_nodes_now, done_edges_now = cytoscape_utils.highlight_path(p, JA_COLOUR, skip_nodes=done_nodes, skip_edges=done_edges)
    done_nodes += done_nodes_now
    done_edges += done_edges_now

In [ ]:
for p in SA_paths:
    done_nodes_now, done_edges_now = cytoscape_utils.highlight_path(p, SA_COLOUR, skip_nodes=done_nodes, skip_edges=done_edges)
    done_nodes += done_nodes_now
    done_edges += done_edges_now

At this point, the Cytoscape session has a network view of the filtered PSS, and highlighting of the paths we extracted from our targeted searches. 

## Extract subnetworks in Cytoscape

Properly inspecting the identified paths is a bit hard within the complete network, so here we pull out the subnetworks of and surrounding the paths. 


### The edge induced network
The first, and smallest subnetwork, is created by extracting only the edges that are present on the paths. 

In [ ]:
network_edge_induced_suid = cytoscape_utils.subnetwork_edge_induced_from_paths(
    paths=JA_paths + SA_paths + ROS_paths,
    g=g,
    parent_suid=pss_network_suid,
    name="identified paths (edge induced)",
)

We apply a new layout to this subnetwork

In [ ]:
_ = p4c.layouts.layout_network('cose', network=network_edge_induced_suid)

### The node induced network

Now we extract the network based on the nodes along the paths, meaning any edges between those nodes that are not on the paths are also extracted. 

In [ ]:
nodes = list(set([y for x in JA_paths + SA_paths + ROS_paths for y in x]))

In [ ]:
network_node_induced_suid = cytoscape_utils.subnetwork_node_induced(
    nodes=nodes,
    parent_suid=pss_network_suid,
    name="identified paths (node induced)",
)

Instead of applying a network layout algorithm, we can copy the layout from the previous subnetwork. 

In [ ]:
_ = p4c.layouts.layout_copycat(
    network_edge_induced_suid, 
    network_node_induced_suid
)

### Neighbours

For more context around our paths, we can include the first neighbours in the view. We can use the Cytoscape first neighbour selection functionality. 

In [ ]:
network_neighbours_suid = cytoscape_utils.subnetwork_neighbours(
    nodes=nodes,
    parent_suid=pss_network_suid,
    name="identified paths + 1st neighbours",
)

In [ ]:
_ = p4c.layouts.layout_network('cose', network=network_neighbours_suid)

### Additional filtering of the neighbours

There are many neighbours displayed now, and we are perhaps only interested in the ones that are connected to at least two of the original path nodes, so we can make a filter using networkX neighbour functions. 

In [ ]:
filtered_neighbours = []
for n in g.nodes():
    if (len([x for x in nx.MultiGraph(g).neighbors(n) if (x in done_nodes)]) > 1) and (n not in done_nodes):
        filtered_neighbours.append(n)

In [ ]:
network_neighbours_filtered_suid = cytoscape_utils.subnetwork_node_induced(
    nodes=nodes+filtered_neighbours,
    parent_suid=pss_network_suid,
    name="identified paths + 1st neighbours (filtered)",
)

In [ ]:
p4c.layouts.layout_copycat(
    network_neighbours_suid, 
    network_neighbours_filtered_suid
)

### Exporting the subnetwork

In [ ]:
g_subgraph_node_induced = nx.induced_subgraph(g, nodes)
print(f"\nNumber of nodes: {g_subgraph_node_induced.number_of_nodes()}\nNumber of edges: {g_subgraph_node_induced.number_of_edges()}")

In [ ]:
nx.to_pandas_edgelist(g_subgraph_node_induced)\
    .to_csv(output_dir / f"pss-dinar-subgraph-refined-edgelist-{today}.tsv", sep="\t", index=None)

## Saving the session

Save the Cytoscape session:

In [ ]:
p4c.session.save_session(str(output_dir / f"PSS-DiNAR-JA-SA-ROS-{today}.cys"))

## Exporting the Cytoscape networks to PDF

In [ ]:
collection_suid = p4c.get_collection_suid(network_edge_induced_suid)

In [ ]:
from skm_tools import cytoscape_pdf_utils

In [ ]:
cytoscape_pdf_utils.export_collection_to_pdfs(collection_suid, output_dir / "figures")

In [ ]:
cytoscape_pdf_utils.export_collection_to_single_pdf(collection_suid, output_dir / "figures" / "single_pdf", caption=True)

In [ ]:
# END